In [1]:
import os
os.environ['RPPREFIX'] = r'C:/Program Files (x86)/REFPROP'
import mat_properties as prop
import numpy as n
import pandas as pd
import cotel
import time
#import Properties as propcoolprop

start_time = time.time()
#-------------input
gasmix= "Nitrogen*Oxygen*CO2*Water*Argon"

RP = prop.init_REFPROP(r'C:\Program Files (x86)\REFPROP')
#---------------------------------------Часть Максима

#Таблица номинального режима
gas_streams0   = pd.read_excel('streams0.xlsx', sheet_name='gas',  index_col=0)
water_streams0 = pd.read_excel('streams0.xlsx', sheet_name='water',index_col=0)
#Таблица частичной нагрузки (входные данные)
gas_streams    = pd.read_excel('streams.xlsx',  sheet_name='gas',  index_col=0)
water_streams  = pd.read_excel('streams.xlsx',  sheet_name='water',index_col=0)


fractiongas0=(gas_streams0.at['GTU-PEVD','N2'],gas_streams0.at['GTU-PEVD','O2'],
                        gas_streams0.at['GTU-PEVD','CO2'],gas_streams0.at['GTU-PEVD','H2O'],gas_streams0.at['GTU-PEVD','Ar'])
fractiongas=(gas_streams.at['GTU-PEVD','N2'],gas_streams.at['GTU-PEVD','O2'],
                        gas_streams.at['GTU-PEVD','CO2'],gas_streams.at['GTU-PEVD','H2O'],gas_streams.at['GTU-PEVD','Ar'])



gas1 = prop.Materials_prop(gasmix,fractiongas, prop.REFPROP_h_s,
                                              prop.REFPROP_p_t,
                                              prop.REFPROP_p_h,
                                              prop.REFPROP_p_s,
                                              prop.REFPROP_p_q,
                                              prop.REFPROP_t_q,
                                              RP=RP)

gas0 = prop.Materials_prop(gasmix,fractiongas0, prop.REFPROP_h_s,
                                              prop.REFPROP_p_t,
                                              prop.REFPROP_p_h,
                                              prop.REFPROP_p_s,
                                              prop.REFPROP_p_q,
                                              prop.REFPROP_t_q,
                                              RP=RP)

water=prop.Materials_prop('water',[1.0,0,0,0,0], prop.REFPROP_h_s,
                                              prop.REFPROP_p_t,
                                              prop.REFPROP_p_h,
                                              prop.REFPROP_p_s,
                                              prop.REFPROP_p_q,
                                              prop.REFPROP_t_q,
                                              RP=RP)

#---------------------------------------Часть Игоря
calctolerance=10**-3
calcmethod = "hybr"

KPD=0.99
iter1=50
Qq = n.zeros(iter1)
for j in range(iter1):
    iter2=50
    Qqq = n.zeros(iter2)
    Ddd = n.zeros(iter2)
    for i in range(iter2):
        PEVD = cotel.heatex('GTU-PEVD','PEVD-IVD','IVD-PEVD','PEVD-DROS',KPD, calctolerance,gas0,gas1,water,
                   calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
        gas_streams.loc['PEVD-IVD','T':'G'] = [PEVD[0],PEVD[1],PEVD[2],PEVD[3]]
        water_streams.loc['PEVD-DROS','T':'G'] = [PEVD[4],PEVD[5],PEVD[6],PEVD[7]]
        IVD  = cotel.vapor ('PEVD-IVD','IVD-EVD','EVD-IVD','IVD-PEVD',KPD,calctolerance,gas1,gas0,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
        gas_streams.loc['IVD-EVD','T':'G'] = [IVD[0],IVD[1],IVD[2],IVD[3]]
        water_streams.loc['IVD-PEVD','T':'G'] = [IVD[4],IVD[5],IVD[6],IVD[7]]
        water_streams.loc['PEVD-DROS':'PEN-EVD','G'] = IVD[7]
        Qqq[i] = PEVD[8]+IVD[8]
        Ddd[i] = water_streams.at['PEVD-DROS','G']
        if i>0:
            if abs((Qqq[i-1]-Qqq[i])/Qqq[i-1]) < calctolerance and abs((Ddd[i-1]-Ddd[i])/Ddd[i-1]) < calctolerance:
                break;
    EVD = cotel.heatex('IVD-EVD','EVD-PPND','PEN-EVD','EVD-IVD',KPD, calctolerance,gas0,gas1,water,
                   calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
    gas_streams.loc['EVD-PPND','T':'G'] = [EVD[0],EVD[1],EVD[2],EVD[3]]
    water_streams.loc['EVD-IVD','T':'G'] = [EVD[4],EVD[5],EVD[6],EVD[7]]
    Qq[j] = PEVD[8]+IVD[8]+EVD[8]
    if j>0:
        if abs((Qq[j-1]-Qq[j])/Qq[j-1]) < calctolerance:
            break;
# print(prop.water_Q_HP(water_streams.at['EVD-IVD','H'],water_streams.at['EVD-IVD','P'])) #сухость пара в экономайзере
# Qgas = KPD*gas_streams.at['GTU-PEVD','G']*(gas_streams.at['GTU-PEVD','H']-gas_streams.at['IVD-EVD','H'])
# Qwat = water_streams.at['PEVD-DROS','G']*(water_streams.at['PEVD-DROS','H']-water_streams.at['EVD-IVD','H'])
# print(Qgas-Qwat)
# print(water_streams)
print("--- %s сек. ---" % round((time.time() - start_time)))



--- 7 сек. ---


In [2]:
gas_streams

,T,P,H,G,N2,O2,CO2,H2O,Ar
GTU-PEVD,522.800000,0.1,936.681566,516.2,0.7803,0.1237,0.0301,0.0594,0.0065
PEVD-IVD,456.173544,0.1,860.833337,516.2,0.7803,0.1237,0.0301,0.0594,0.0065
IVD-EVD,308.683794,0.1,697.102746,516.2,0.7803,0.1237,0.0301,0.0594,0.0065
EVD-PPND,235.394570,0.1,617.793172,516.2,0.7803,0.1237,0.0301,0.0594,0.0065
PPND-IND,NaN,NaN,NaN,NaN,0.7803,0.1237,0.0301,0.0594,0.0065
IND-GPK,175.900000,0.1,554.284799,516.2,0.7803,0.1237,0.0301,0.0594,0.0065
GPK-out,NaN,NaN,NaN,NaN,0.7803,0.1237,0.0301,0.0594,0.0065


In [3]:
water_streams

,T,P,H,G
PEVD-DROS,496.959461,8.012386,3391.805379,60.715862
IVD-PEVD,297.938037,8.341200,2753.507839,60.715862
EVD-IVD,297.938037,8.341200,1375.194327,60.715862
PEN-EVD,166.400000,8.341200,707.655323,60.715862
BND-PEN,NaN,NaN,NaN,NaN
PPND-TURB,NaN,NaN,NaN,NaN
BND-PPND,NaN,NaN,NaN,NaN
IND-BND,NaN,NaN,NaN,NaN
BND-IND,NaN,NaN,NaN,NaN
GPK-BND,NaN,NaN,NaN,NaN
